# Resultados gerais

In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
from generate_tsp import GerarProblemaRadialTSP
from sga import SGAPermutacao, SGAInteiroLimitado

In [3]:
def EscreverResultado(res,params,topologia,tipo,vetorOtimo):
    params['Topologia'] = topologia
    pop,stats,hof,params,custo = res
    resPerm = pd.Series(params)
    resPerm['Tipo'] = tipo
    minVal, gap, success, optSuccess = stats.select("min",'gap','success','optimal success')
    resPerm['Sucesso fraco'] = np.any(success)
    resPerm['Sucesso'] = np.any(optSuccess)
    resPerm['Geração final fraco'] = np.argwhere(success)[0][0] if np.any(success) else len(success)
    resPerm['Geração final'] = np.argwhere(optSuccess)[0][0] if np.any(optSuccess) else len(optSuccess)
    resPerm['MBF'] = np.mean(minVal)
    resPerm['Valor encontrado'] = hof[0].fitness.values[0]
    resPerm['Gap'] = custo(vetorOtimo)[0]/resPerm['Valor encontrado']
    resPerm['Indivíduo encontrado'] = hof[0]
    return pd.DataFrame([resPerm])
    
def Experimento(posicoes,params,nomePosicao='Padrão',vetorOtimo=None):
    res = SGAPermutacao(posicoes,params,vetorOtimo=vetorOtimo)
    resPerm = EscreverResultado(res,params,nomePosicao,'Permutação',vetorOtimo)
    res = SGAInteiroLimitado(posicoes,params,vetorOtimo=vetorOtimo)
    resInt = EscreverResultado(res,params,nomePosicao,'Inteiro',vetorOtimo)
    return pd.concat([resPerm,resInt],ignore_index=True)

In [4]:
def CalcularSR(resultados):
    return (100*resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso'].sum()/resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso'].count()).unstack(['Topologia',"Tipo"])
CalcularMBF = lambda resultados: resultados.groupby(['Topologia','Tipo','P','mu'])['MBF'].describe().reset_index().set_index(['Topologia','Tipo','P','mu']).unstack(['Topologia','Tipo'])
CalcularAES = lambda resultados: resultados[resultados['Sucesso'] == True].groupby(['Topologia','Tipo','P','mu'])['Geração final'].describe().unstack(['Topologia','Tipo'])
def CalcularSRFraco(resultados):
    return (100*resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso fraco'].sum()/resultados.groupby(['Topologia','Tipo','P','mu'])['Sucesso fraco'].count()).unstack(['Topologia',"Tipo"])
CalcularAES = lambda resultados: resultados[resultados['Sucesso fraco'] == True].groupby(['Topologia','Tipo','P','mu'])['Geração final fraco'].describe().unstack(['Topologia','Tipo'])

In [5]:
resultados = pd.DataFrame()

In [7]:
from itertools import product
from tqdm.notebook import trange, tqdm

dbconn = sqlite3.connect("Resultados-finais.sqlite")

cidadesLista = [10,20,30]
individuosLista = [20,50,100]
repeticoes = 100
geracoes = 1000

listaExecucoes = list(product(list(range(repeticoes)),cidadesLista,individuosLista))
np.random.shuffle(listaExecucoes)

for repeticoes, cidades, individuos in tqdm(listaExecucoes):
    resultados = pd.DataFrame()
    posicoesNormal = GerarProblemaRadialTSP(cidades)
    posicoesAleatorio = posicoesNormal.copy()
    
    resultadoOtimo = np.arange(posicoesNormal.shape[1])
    embaralhamento = resultadoOtimo.copy()
    
    np.random.shuffle(embaralhamento)
    posicoesAleatorio = posicoesNormal[:,embaralhamento].copy()
    resultadoOtimoAleatorio = np.zeros(posicoesAleatorio.shape[1], dtype=int)
    for i, val in enumerate(embaralhamento):
        resultadoOtimoAleatorio[val] = i

    params = {
        "N":geracoes,
        "P":posicoesNormal.shape[1],
        "mu":individuos,
        "cxpb": 0.7,
        "mupb":0.2,
        "muidxpb":0.05
    }
    resultados = pd.concat([resultados,Experimento(posicoesNormal,params,'Ordenada',vetorOtimo=resultadoOtimo)],ignore_index=True)
    resultados = pd.concat([resultados,Experimento(posicoesAleatorio,params,'Aleatória',vetorOtimo=resultadoOtimoAleatorio)],ignore_index=True)
    resultados.to_sql("resultados",dbconn,if_exists='append')

  0%|          | 0/900 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
CalcularSRFraco(resultados)

Topologia  Aleatória                Ordenada            
Tipo         Inteiro  Permutação     Inteiro  Permutação
P  mu                                                   
11 20       0.000000   66.666667  100.000000   66.666667
   50      33.333333   66.666667  100.000000  100.000000
   100     33.333333  100.000000  100.000000  100.000000
21 20       0.000000    0.000000  100.000000    0.000000
   50       0.000000    0.000000  100.000000   66.666667
   100      0.000000   33.333333  100.000000   33.333333
31 20       0.000000    0.000000   66.666667    0.000000
   50       0.000000    0.000000  100.000000    0.000000
   100      0.000000    0.000000  100.000000    0.000000

In [8]:
resultados[(resultados['Gap']>0.9) & (resultados['Sucesso']==False)]

,N,P,mu,cxpb,mupb,muidxpb,Topologia,Tipo,Sucesso fraco,Sucesso,Geração final fraco,Geração final,MBF,Valor encontrado,Gap,Indivíduo encontrado
58,1000,21,100,0.4,0.1,0.01,Aleatória,Permutação,True,False,517,1001,8.628282,6.867709,0.911130,"[17, 11, 13, 18, 3, 16, 10, 19, 7, 1, 8, 12, 5..."
97,1000,31,20,0.4,0.1,0.01,Ordenada,Inteiro,True,False,813,1001,11.146226,6.685241,0.938142,"[28, 30, 0, 28, 30, 0, 0, 29, 1, 30, 2, 29, 30..."


In [9]:
CalcularMBF(resultados)

count                                      mean             \
Topologia Aleatória            Ordenada             Aleatória              
Tipo        Inteiro Permutação  Inteiro Permutação    Inteiro Permutação   
P  mu                                                                      
11 20           3.0        3.0      3.0        3.0   9.510704   7.829732   
   50           3.0        3.0      3.0        3.0   9.184132   7.691209   
   100          3.0        3.0      3.0        3.0   7.995558   6.714215   
21 20           3.0        3.0      3.0        3.0  18.017605  14.531879   
   50           3.0        3.0      3.0        3.0  15.574497  11.709545   
   100          3.0        3.0      3.0        3.0  15.226324  10.512177   
31 20           3.0        3.0      3.0        3.0  25.257286  19.780216   
   50           3.0        3.0      3.0        3.0  22.903229  16.719013   
   100          3.0        3.0      3.0        3.0  19.745471  13.128365   

                                      std             ...        50%  \
Topologia   Ordenada            Aleatória             ...   Ordenada   
Tipo         Inteiro Permutação   Inteiro Permutação  ...    Inteiro   
P  mu                                                 ...              
11 20       6.364575   7.994896  0.463718   1.326104  ...   6.317357   
   50       6.204639   7.025773  0.752743   1.691901  ...   6.196305   
   100      6.186429   6.438079  1.932006   0.559229  ...   6.189252   
21 20       8.066264  14.669930  1.305857   1.160959  ...   8.013672   
   50       6.733017  10.138570  0.817025   0.996699  ...   6.760902   
   100      6.390259  10.285973  1.930116   1.681138  ...   6.395253   
31 20      10.981806  20.771171  1.534169   0.858987  ...  11.146226   
   50       7.713351  16.448050  3.205557   0.766084  ...   7.521128   
   100      7.268811  15.730420  3.955736   0.950515  ...   7.064491   

                            75%                                         max  \
Topologia             Aleatória              Ordenada             Aleatória   
Tipo      Permutação    Inteiro Permutação    Inteiro Permutação    Inteiro   
P  mu                                                                         
11 20       7.529628   9.776728   8.356077   6.453858   8.366640   9.828950   
   50       6.945352   9.537155   8.321501   6.215176   7.099266   9.976184   
   100      6.467761   8.828137   6.906357   6.189474   6.474510  10.113859   
21 20      14.081278  18.770470  15.146588   8.260860  15.010267  18.840076   
   50       9.184813  16.044081  12.284596   6.873625  10.859834  16.121561   
   100     11.368856  15.976053  11.454125   6.434440  11.662553  17.404013   
31 20      21.036881  26.115656  20.218135  12.869508  21.166630  26.494176   
   50      16.125882  24.573778  17.116088   8.004504  16.975855  25.953413   
   100     15.375273  22.025353  13.677145   7.393143  16.284307  22.258285   

                                            
Topologia              Ordenada             
Tipo      Permutação    Inteiro Permutação  
P  mu                                       
11 20       9.319111   6.590360   9.203652  
   50       9.614066   6.234047   7.253181  
   100      7.355780   6.189697   6.481259  
21 20      15.609432   8.508048  15.939256  
   50      12.321447   6.986348  12.534855  
   100     11.859624   6.473627  11.956251  
31 20      20.621297  14.592791  21.296380  
   50      17.453547   8.487881  17.825827  
   100     13.677247   7.721794  17.193341  

[9 rows x 32 columns]

In [10]:
CalcularAES(resultados)

count                                     mean             \
Topologia Aleatória            Ordenada            Aleatória              
Tipo        Inteiro Permutação  Inteiro Permutação   Inteiro Permutação   
P  mu                                                                     
11 20           NaN        2.0      3.0        2.0       NaN      598.5   
   50           1.0        2.0      3.0        3.0     833.0      236.0   
   100          1.0        3.0      3.0        3.0      96.0      194.0   
21 20           NaN        NaN      3.0        NaN       NaN        NaN   
   50           NaN        NaN      3.0        2.0       NaN        NaN   
   100          NaN        1.0      3.0        1.0       NaN      517.0   
31 20           NaN        NaN      2.0        NaN       NaN        NaN   
   50           NaN        NaN      3.0        NaN       NaN        NaN   
   100          NaN        NaN      3.0        NaN       NaN        NaN   

                                        std              ...      50%  \
Topologia    Ordenada             Aleatória              ... Ordenada   
Tipo          Inteiro  Permutação   Inteiro  Permutação  ...  Inteiro   
P  mu                                                    ...            
11 20      114.000000  582.000000       NaN  204.353860  ...    123.0   
   50       14.666667  279.666667       NaN  148.492424  ...      7.0   
   100       4.000000  152.666667       NaN  138.358231  ...      4.0   
21 20      382.000000         NaN       NaN         NaN  ...    405.0   
   50      154.000000  656.500000       NaN         NaN  ...    165.0   
   100      40.333333  240.000000       NaN         NaN  ...     37.0   
31 20      531.000000         NaN       NaN         NaN  ...    531.0   
   50      345.666667         NaN       NaN         NaN  ...    215.0   
   100     205.666667         NaN       NaN         NaN  ...    128.0   

                           75%                                      max  \
Topologia            Aleatória            Ordenada            Aleatória   
Tipo      Permutação   Inteiro Permutação  Inteiro Permutação   Inteiro   
P  mu                                                                     
11 20          582.0       NaN     670.75    170.0     667.50       NaN   
   50          284.0     833.0     288.50     21.0     299.50     833.0   
   100         147.0      96.0     240.50      6.0     188.50      96.0   
21 20            NaN       NaN        NaN    409.5        NaN       NaN   
   50          656.5       NaN        NaN    207.5     670.25       NaN   
   100         240.0       NaN     517.00     55.5     240.00       NaN   
31 20            NaN       NaN        NaN    672.0        NaN       NaN   
   50            NaN       NaN        NaN    469.0        NaN       NaN   
   100           NaN       NaN        NaN    273.5        NaN       NaN   

                                          
Topologia            Ordenada             
Tipo      Permutação  Inteiro Permutação  
P  mu                                     
11 20          743.0    217.0      753.0  
   50          341.0     35.0      315.0  
   100         353.0      8.0      230.0  
21 20            NaN    414.0        NaN  
   50            NaN    250.0      684.0  
   100         517.0     74.0      240.0  
31 20            NaN    813.0        NaN  
   50            NaN    723.0        NaN  
   100           NaN    419.0        NaN  

[9 rows x 32 columns]

In [ ]:
gen, avg, min, success, gap, pop = logbook.select('gen','mean','min','success','gap','pop')

fig, ax = plt.subplots()
ax.plot(gen,avg,label="Média")
ax.plot(gen,min,label="Menor")
ax.plot(gen,success,label="Sucesso")
ax.legend()
ax.title("Estatísticas através das gerações")
plt.show()

print(np.array(pop).shape)
for i in range(np.array(pop).shape[1]):
    plt.plot(gen,np.array(pop)[:,i,0])
plt.title("Aptidões individuais")
plt.show()